In [ ]:
#! /usr/lib/python
# -*- coding: utf-8 -*-

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.externals import joblib

In [ ]:
# データの読み込み
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
n_samples, h, w = lfw_people.images.shape
X = lfw_people.data
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("データセット概要:")
print("データ数: %d " % n_samples)
print("次元数: %d " % X.shape[1])
print("クラスタ数: %d " % n_classes)
print()

print("ターゲット:")
print(X)
print()

print("ラベル:")
print(y)
print()

In [ ]:
# 学習用とテスト用へとデータを分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# 主成分分析
n_components = 150
pca = PCA(svd_solver='randomized', n_components=n_components, whiten=True).fit(X_train)
joblib.dump(pca, 'pca.pkl', compress = 1)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
# SVCのパラメータチューニング
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train_pca, y_train)

In [ ]:
print("解析器パラメータ:")
print(clf.best_estimator_)
print()

In [ ]:
# テストデータを用いた評価
y_pred = clf.predict(X_test_pca)

In [ ]:
print("解析結果:")
print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))
print()

In [ ]:
# 分類器のエクスポート
joblib.dump(clf.best_estimator_, 'svc.pkl', compress = 1)